# Импортируем инструменты

In [ ]:
from google.auth import default
from google.colab import auth, files
import gspread
import json
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import requests as req

In [ ]:
# Авторизуемся на Google Cloud, чтобы использовать базу данных BigQuery
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
%%capture
!pip install pymorphy2

In [ ]:
%%capture
!pip install nltk

In [ ]:
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from pymorphy2 import MorphAnalyzer
import pymorphy2

In [ ]:
%%capture
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0.tar.gz

In [ ]:
%%capture
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz

# Заливаем посты в Google-таблицу

In [ ]:
uploaded = files.upload()

Saving zen_of_python.json to zen_of_python.json


In [ ]:
data = next(iter(uploaded.values()))

In [ ]:
posts = json.loads(data.decode())
print(posts[725])

{'id': 2261, 'date': '2023-03-11 11:39:00+00:00', 'message': 'Объяснение: \n\nЗначение varTwo передаётся в функцию func(). varOne[1, 2, 3] передаётся туда же как ещё одно значение\n\nПри этом внутри функции arr[0] = 0 изменяет значение по индексу [0] в списке. Отсюда следует, что varOne = [0,2,3]', 'views': 4938, 'forwards': 3, 'replies': 0, 'reactions': [{'emojis': '👍', 'emoji_count': 8}, {'emojis': '🤔', 'emoji_count': 4}]}


In [ ]:
try:
  list1 = [item for item in posts if item != None]
  print(len(list1))
except AttributeError as e:
  print(e)

726


In [ ]:
df = pd.DataFrame(list1)
print(len(df))

726


In [ ]:
spreadsheet_key = 'spreadsheet_key'
workbook = gc.open_by_key(spreadsheet_key)

# Подготовим датасет согласно требованиям Google API
# Конвертируем временные штампы в строки
df = df.astype(str)

# Обновим таблицу
workbook.values_update(
  # Укажем диапазон
  'A1:Z20000',
  params = {
      'valueInputOption': 'USER_ENTERED'
  },
  body = {
      'values': [df.columns.values.tolist()] + df.values.tolist()
  }
)

{'spreadsheetId': '1lzyEy3rNAamdlfsvVaYDsurNcXusrX3U3S2YmxVlroY',
 'updatedRange': "'Лист1'!A1:G727",
 'updatedRows': 727,
 'updatedColumns': 7,
 'updatedCells': 5089}

# Cамые популярные эмоджи

In [ ]:
# Подгрузим посты
table = gc.open_by_url('url')
sheet = table.sheet1
posts = pd.DataFrame(sheet.get_all_records())
posts.head()

,id,date,message,views,forwards,replies,reactions
0,2833,2023-11-26 07:12:04+00:00,,4220,25,6,"[{'emojis': '😁', 'emoji_count': 52}, {'emojis'..."
1,2130,2023-01-11 16:33:02+00:00,​Я: да не нужны мне комментарии к коду. Я не з...,7037,72,5,"[{'emojis': '😁', 'emoji_count': 87}, {'emojis'..."
2,2560,2023-08-03 06:11:02+00:00,​Обработка больших файлов в Python с использов...,4539,86,0,"[{'emojis': '🔥', 'emoji_count': 6}]"
3,2425,2023-06-09 05:14:03+00:00,​Как превратить скрипт.py в программу.exe\n\nУ...,5529,158,10,"[{'emojis': '👍', 'emoji_count': 6}, {'emojis':..."
4,2649,2023-09-08 06:22:03+00:00,​String Template: ещё один способ интерполиров...,4500,19,2,"[{'emojis': '👍', 'emoji_count': 12}, {'emojis'..."


In [ ]:
# Выделим посты за год
last_year_posts = posts[(posts['date'] >= '2023-01-01') & (posts['date'] <= '2023-12-31')]

In [ ]:
posts = last_year_posts.copy()

In [ ]:
posts['👍'] = posts['reactions'].str.extract("emojis': '👍', 'emoji_count': (.+?)}")
posts['⚡'] = posts['reactions'].str.extract("emojis': '⚡', 'emoji_count': (.+?)}")
posts['❤'] = posts['reactions'].str.extract("emojis': '❤', 'emoji_count': (.+?)}")
posts['🍌'] = posts['reactions'].str.extract("emojis': '🍌', 'emoji_count': (.+?)}")
posts['🎉'] = posts['reactions'].str.extract("emojis': '🎉', 'emoji_count': (.+?)}")
posts['🐳'] = posts['reactions'].str.extract("emojis': '🐳', 'emoji_count': (.+?)}")
posts['👌'] = posts['reactions'].str.extract("emojis': '👌', 'emoji_count': (.+?)}")
posts['👎'] = posts['reactions'].str.extract("emojis': '👎', 'emoji_count': (.+?)}")
posts['👏'] = posts['reactions'].str.extract("emojis': '👏', 'emoji_count': (.+?)}")
posts['💋'] = posts['reactions'].str.extract("emojis': '💋', 'emoji_count': (.+?)}")
posts['💩'] = posts['reactions'].str.extract("emojis': '💩', 'emoji_count': (.+?)}")
posts['💯'] = posts['reactions'].str.extract("emojis': '💯', 'emoji_count': (.+?)}")
posts['🔥'] = posts['reactions'].str.extract("emojis': '🔥', 'emoji_count': (.+?)}")
posts['🗿'] = posts['reactions'].str.extract("emojis': '🗿', 'emoji_count': (.+?)}")
posts['🤔'] = posts['reactions'].str.extract("emojis': '🤔', 'emoji_count': (.+?)}")
posts['🤡'] = posts['reactions'].str.extract("emojis': '🤡', 'emoji_count': (.+?)}")
posts['🤣'] = posts['reactions'].str.extract("emojis': '🤣', 'emoji_count': (.+?)}")
posts['🤨'] = posts['reactions'].str.extract("emojis': '🤨', 'emoji_count': (.+?)}")
posts['🤬'] = posts['reactions'].str.extract("emojis': '🤬', 'emoji_count': (.+?)}")
posts['🤮'] = posts['reactions'].str.extract("emojis': '🤮', 'emoji_count': (.+?)}")
posts['🤯'] = posts['reactions'].str.extract("emojis': '🤯', 'emoji_count': (.+?)}")
posts['🥰'] = posts['reactions'].str.extract("emojis': '🥰', 'emoji_count': (.+?)}")
posts['🥱'] = posts['reactions'].str.extract("emojis': '🥱', 'emoji_count': (.+?)}")
posts['🥴'] = posts['reactions'].str.extract("emojis': '🥴', 'emoji_count': (.+?)}")
posts['🫡'] = posts['reactions'].str.extract("emojis': '🫡', 'emoji_count': (.+?)}")
posts['😁'] = posts['reactions'].str.extract("emojis': '😁', 'emoji_count': (.+?)}")
posts['😐'] = posts['reactions'].str.extract("emojis': '😐', 'emoji_count': (.+?)}")
posts['😱'] = posts['reactions'].str.extract("emojis': '😱', 'emoji_count': (.+?)}")

posts.sample(10)

,id,date,message,views,forwards,replies,reactions,👍,⚡,❤,...,🤬,🤮,🤯,🥰,🥱,🥴,🫡,😁,😐,😱
17,2126,2023-01-09 13:33:02+00:00,​Как написать трекинг парковочных мест\n\nВам ...,6968,110,4,"[{'emojis': '👏', 'emoji_count': 15}, {'emojis'...",NaN,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,2272,2023-03-16 11:01:02+00:00,​Дайджест Python #5: как сделать свой Copilot ...,5540,33,2,"[{'emojis': '❤', 'emoji_count': 5}, {'emojis':...",NaN,NaN,5,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,2270,2023-03-14 18:47:08+00:00,,5833,43,0,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,2268,2023-03-14 18:47:08+00:00,OpenAI представила GPT-4 и расскажет о ней под...,5141,45,0,"[{'emojis': '👍', 'emoji_count': 18}, {'emojis'...",18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,2631,2023-09-01 10:00:03+00:00,"А что выберете вы?\n\nНе знаете, что из этого ...",4207,1,2,"[{'emojis': '💩', 'emoji_count': 23}, {'emojis'...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,2853,2023-12-04 15:02:49+00:00,​​Асинхронность для начинающих\n\nТрёхчасовой ...,6757,198,1,"[{'emojis': '🔥', 'emoji_count': 7}, {'emojis':...",NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678,2778,2023-11-01 15:22:03+00:00,​whoogle search для поиска без рекламы\n\nТако...,5223,17,1,"[{'emojis': '👍', 'emoji_count': 3}, {'emojis':...",3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565,2626,2023-08-30 14:33:04+00:00,,4792,5,0,"[{'emojis': '😁', 'emoji_count': 33}, {'emojis'...",1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,NaN,NaN
664,2447,2023-06-17 15:43:03+00:00,​Оператор xor\n\nВ Python оператор xor (исключ...,4587,24,14,"[{'emojis': '👍', 'emoji_count': 17}, {'emojis'...",17,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,2588,2023-08-14 06:12:03+00:00,​Как насчёт теста на Data Engineer?\n\nSpark я...,4642,29,2,"[{'emojis': '👍', 'emoji_count': 2}, {'emojis':...",2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Вычислим самый комментируемый пост
most_replied_post = posts.iloc[posts['replies'].idxmax()]
link = 'https://t.me/zen_of_python/' + str(most_replied_post['id'])
print(link)

https://t.me/zen_of_python/2255


In [ ]:
posts = posts.apply(pd.to_numeric, errors='coerce')

posts.loc['total'] = posts.sum(numeric_only=True)
posts.tail()

,id,date,message,views,forwards,replies,reactions,👍,⚡,❤,...,🤬,🤮,🤯,🥰,🥱,🥴,🫡,😁,😐,😱
722,2774.0,NaN,NaN,4478.0,8.0,2.0,NaN,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,2324.0,NaN,NaN,5404.0,48.0,3.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,2631.0,NaN,NaN,4207.0,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,2261.0,NaN,NaN,4938.0,3.0,0.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total,1814199.0,0.0,0.0,4009046.0,38285.0,3166.0,0.0,4550.0,21.0,798.0,...,10.0,116.0,45.0,29.0,8.0,5.0,0.0,1485.0,15.0,6.0


In [ ]:
# \U0001fae1 - это 🫡
totals = posts.tail(1)
totals.to_dict()

{'id': {'total': 1814199.0},
 'date': {'total': 0.0},
 'message': {'total': 0.0},
 'views': {'total': 4009046.0},
 'forwards': {'total': 38285.0},
 'replies': {'total': 3166.0},
 'reactions': {'total': 0.0},
 '👍': {'total': 4550.0},
 '⚡': {'total': 21.0},
 '❤': {'total': 798.0},
 '🍌': {'total': 188.0},
 '🎉': {'total': 34.0},
 '🐳': {'total': 2.0},
 '👌': {'total': 46.0},
 '👎': {'total': 610.0},
 '👏': {'total': 108.0},
 '💋': {'total': 0.0},
 '💩': {'total': 1101.0},
 '💯': {'total': 7.0},
 '🔥': {'total': 753.0},
 '🗿': {'total': 16.0},
 '🤔': {'total': 184.0},
 '🤡': {'total': 262.0},
 '🤣': {'total': 176.0},
 '🤨': {'total': 13.0},
 '🤬': {'total': 10.0},
 '🤮': {'total': 116.0},
 '🤯': {'total': 45.0},
 '🥰': {'total': 29.0},
 '🥱': {'total': 8.0},
 '🥴': {'total': 5.0},
 '\U0001fae1': {'total': 0.0},
 '😁': {'total': 1485.0},
 '😐': {'total': 15.0},
 '😱': {'total': 6.0}}

# Самый популярный хэштег

In [ ]:
posts['message'] = posts['message'].astype('str')
posts['hashtags'] = posts['message'].str.findall(r'#.*?(?=\s|$)')
posts.sample(20)

,id,date,message,views,forwards,replies,reactions,hashtags
11,2171,2023-01-27 11:01:01+00:00,​Хотите вырасти в разработке на Python? Без но...,6290,10,0,"[{'emojis': '💩', 'emoji_count': 6}, {'emojis':...",[]
269,2771,2023-10-30 14:04:08+00:00,,4623,14,0,[],[]
412,2354,2023-05-01 13:40:12+00:00,​Сколько рядов набрали?,5489,10,3,"[{'emojis': '😁', 'emoji_count': 26}, {'emojis'...",[]
523,2338,2023-04-22 10:01:03+00:00,,5433,0,15,"[{'emojis': '🤣', 'emoji_count': 21}, {'emojis'...",[]
321,2814,2023-11-17 15:01:02+00:00,​Пятничный Win Share\n\nЧто интересного у вас ...,4351,1,6,"[{'emojis': '🍌', 'emoji_count': 1}]",[#winshare]
16,2576,2023-08-09 06:12:03+00:00,​Как бороться с прокрастинацией\n\nНеобычная а...,4825,34,8,"[{'emojis': '👍', 'emoji_count': 10}]",[#карьера]
12,2521,2023-07-18 06:01:03+00:00,​Причесываем многострочный вывод с print()\n\n...,4541,46,7,"[{'emojis': '👍', 'emoji_count': 23}]",[#лучшиепрактики]
540,2400,2023-05-31 05:31:03+00:00,​Что такое Zen of Python?\n\nЭто набор принцип...,5028,30,0,"[{'emojis': '👍', 'emoji_count': 17}, {'emojis'...",[#лучшиепрактики]
60,2716,2023-10-08 07:11:02+00:00,​7 шагов по контейнеризации Python-приложений\...,5563,134,0,"[{'emojis': '❤', 'emoji_count': 5}, {'emojis':...",[#docker]
635,2216,2023-02-18 14:31:01+00:00,,5294,28,0,[],[]


In [ ]:
res = posts['hashtags'].str.split(expand=True).stack().value_counts()
most_popular_hashtags = res.sort_values(ascending=False)
most_popular_hashtags[:20]

[]                     205
['#лучшиепрактики']     52
['#библиотека']         50
['#факты']              44
'#',                    41
['#',                   34
'#=>',                  32
['#задача']             25
'#лучшиепрактики']      19
['#карьера']            16
['#winshare']           15
'#собеседование']       13
['#django']             12
['#=>',                 10
['#обучение']            9
['#начинающим']          8
['#нейросети',           8
'#библиотека']           7
'#telegram']             7
['#книга']               6
dtype: int64

# Самые популярные технологии

In [ ]:
# Подгрузим посты
table = gc.open_by_url('url')
sheet = table.sheet1
comments = pd.DataFrame(sheet.get_all_records())
comments.head()

,reply_to_msg_id,comment_id,date,text,reactions
0,18859,18861,2023-12-22 06:36:16+00:00,Хреновая подборка.\nJSON в CSV надёжнее залить...,None
1,18859,18860,2023-12-22 06:25:35+00:00,"скажите ей уже, наконец",MessageReactions(results=[ReactionCount(reacti...
2,18809,18851,2023-12-20 19:20:06+00:00,Обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...
3,18809,18846,2023-12-20 16:43:20+00:00,Объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None


In [ ]:
comments["text"] = comments['text'].str.replace('[^\w\s]','')
comments.head(10)

<ipython-input-10-f3d81c0ebeb8>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  comments["text"] = comments['text'].str.replace('[^\w\s]','')


,reply_to_msg_id,comment_id,date,text,reactions
0,18859,18861,2023-12-22 06:36:16+00:00,Хреновая подборка\nJSON в CSV надёжнее залить ...,None
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...
2,18809,18851,2023-12-20 19:20:06+00:00,Обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...
3,18809,18846,2023-12-20 16:43:20+00:00,Объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None
5,18853,18854,2023-12-21 05:25:45+00:00,Не ну такое вполне возможно Просто именно для ...,None
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None
7,18825,18852,2023-12-21 04:41:56+00:00,Почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...
8,18795,18850,2023-12-20 16:49:46+00:00,Послушай я очень устал и могу сказать что для...,None
9,18848,18849,2023-12-20 16:47:13+00:00,Так а проект тут причем,None


In [ ]:
# Выполним лемматизацию русских слов
ru_core = spacy.load('ru_core_news_sm')
comments['text'] = comments['text'].astype('str')
comments['text'] = [entry.lower() for entry in comments['text']]

comments["lemmatized_ru"] = comments['text'].apply(lambda x: " ".join([y.lemma_ for y in ru_core(x)]))
comments.head(10)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  directory, to prevent mismatches with relative paths.


,reply_to_msg_id,comment_id,date,text,reactions,lemmatized_ru
0,18859,18861,2023-12-22 06:36:16+00:00,хреновая подборка\njson в csv надёжнее залить ...,None,хреновый подборка \n json в csv надёжнее залит...
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...,сказать ей уже наконец
2,18809,18851,2023-12-20 19:20:06+00:00,обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,обожать его лекция
3,18809,18846,2023-12-20 16:43:20+00:00,объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,объяснять на палец
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None,это классика
5,18853,18854,2023-12-21 05:25:45+00:00,не ну такое вполне возможно просто именно для ...,None,не ну такой вполне возможный просто именно для...
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None,оформление самого гитхаба подписка на проект и...
7,18825,18852,2023-12-21 04:41:56+00:00,почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...,почему за 6 год работа должный быть чтото на г...
8,18795,18850,2023-12-20 16:49:46+00:00,послушай я очень устал и могу сказать что для...,None,послушай я очень устать и мочь сказать что д...
9,18848,18849,2023-12-20 16:47:13+00:00,так а проект тут причем,None,так а проект тут причем


In [ ]:
# Выполним лемматизацию английских слов
en_core = spacy.load('en_core_web_sm')

comments["lemmatized_en"] = comments['text'].apply(lambda x: " ".join([y.lemma_ for y in en_core(x)]))
comments.head(20)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  directory, to prevent mismatches with relative paths.


,reply_to_msg_id,comment_id,date,text,reactions,lemmatized_ru,lemmatized_en
0,18859,18861,2023-12-22 06:36:16+00:00,хреновая подборка\njson в csv надёжнее залить ...,None,хреновый подборка \n json в csv надёжнее залит...,хреновая подборка \n json в csv надёжнее залит...
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...,сказать ей уже наконец,скажите ей уже наконец
2,18809,18851,2023-12-20 19:20:06+00:00,обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,обожать его лекция,обожаю его лекции
3,18809,18846,2023-12-20 16:43:20+00:00,объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,объяснять на палец,объясняет на пальцах
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None,это классика,это классика
5,18853,18854,2023-12-21 05:25:45+00:00,не ну такое вполне возможно просто именно для ...,None,не ну такой вполне возможный просто именно для...,не ну такое вполне возможно просто именно для ...
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None,оформление самого гитхаба подписка на проект и...,оформление самого гитхаба подписки на проекты ...
7,18825,18852,2023-12-21 04:41:56+00:00,почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...,почему за 6 год работа должный быть чтото на г...,почему за 6 лет работы должно быть чтото на ги...
8,18795,18850,2023-12-20 16:49:46+00:00,послушай я очень устал и могу сказать что для...,None,послушай я очень устать и мочь сказать что д...,послушай я очень устал и могу сказать что дл...
9,18848,18849,2023-12-20 16:47:13+00:00,так а проект тут причем,None,так а проект тут причем,так а проект тут причем


In [ ]:
comments['lemmatized_en_clean'] = comments['lemmatized_en'].str.encode('ascii', 'ignore').str.decode('ascii')
comments.head(10)

,reply_to_msg_id,comment_id,date,text,reactions,lemmatized_ru,lemmatized_en,lemmatized_en_clean
0,18859,18861,2023-12-22 06:36:16+00:00,хреновая подборка\njson в csv надёжнее залить ...,None,хреновый подборка \n json в csv надёжнее залит...,хреновая подборка \n json в csv надёжнее залит...,\n json csv csvwriter pandas \n passwo...
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...,сказать ей уже наконец,скажите ей уже наконец,
2,18809,18851,2023-12-20 19:20:06+00:00,обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,обожать его лекция,обожаю его лекции,
3,18809,18846,2023-12-20 16:43:20+00:00,объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,объяснять на палец,объясняет на пальцах,
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None,это классика,это классика,
5,18853,18854,2023-12-21 05:25:45+00:00,не ну такое вполне возможно просто именно для ...,None,не ну такой вполне возможный просто именно для...,не ну такое вполне возможно просто именно для ...,...
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None,оформление самого гитхаба подписка на проект и...,оформление самого гитхаба подписки на проекты ...,\n 6 ...
7,18825,18852,2023-12-21 04:41:56+00:00,почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...,почему за 6 год работа должный быть чтото на г...,почему за 6 лет работы должно быть чтото на ги...,6 \n\n selfhoste ...
8,18795,18850,2023-12-20 16:49:46+00:00,послушай я очень устал и могу сказать что для...,None,послушай я очень устать и мочь сказать что д...,послушай я очень устал и могу сказать что дл...,
9,18848,18849,2023-12-20 16:47:13+00:00,так а проект тут причем,None,так а проект тут причем,так а проект тут причем,


In [ ]:
comments['lemmatized_ru_clean'] = comments['lemmatized_ru'].str.replace("[a-z]", " ");
comments.head(10)

<ipython-input-14-17b33ba22936>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  comments['lemmatized_ru_clean'] = comments['lemmatized_ru'].str.replace("[a-z]", " ");


,reply_to_msg_id,comment_id,date,text,reactions,lemmatized_ru,lemmatized_en,lemmatized_en_clean,lemmatized_ru_clean
0,18859,18861,2023-12-22 06:36:16+00:00,хреновая подборка\njson в csv надёжнее залить ...,None,хреновый подборка \n json в csv надёжнее залит...,хреновая подборка \n json в csv надёжнее залит...,\n json csv csvwriter pandas \n passwo...,хреновый подборка \n в надёжнее залит...
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...,сказать ей уже наконец,скажите ей уже наконец,,сказать ей уже наконец
2,18809,18851,2023-12-20 19:20:06+00:00,обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,обожать его лекция,обожаю его лекции,,обожать его лекция
3,18809,18846,2023-12-20 16:43:20+00:00,объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,объяснять на палец,объясняет на пальцах,,объяснять на палец
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None,это классика,это классика,,это классика
5,18853,18854,2023-12-21 05:25:45+00:00,не ну такое вполне возможно просто именно для ...,None,не ну такой вполне возможный просто именно для...,не ну такое вполне возможно просто именно для ...,...,не ну такой вполне возможный просто именно для...
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None,оформление самого гитхаба подписка на проект и...,оформление самого гитхаба подписки на проекты ...,\n 6 ...,оформление самого гитхаба подписка на проект и...
7,18825,18852,2023-12-21 04:41:56+00:00,почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...,почему за 6 год работа должный быть чтото на г...,почему за 6 лет работы должно быть чтото на ги...,6 \n\n selfhoste ...,почему за 6 год работа должный быть чтото на г...
8,18795,18850,2023-12-20 16:49:46+00:00,послушай я очень устал и могу сказать что для...,None,послушай я очень устать и мочь сказать что д...,послушай я очень устал и могу сказать что дл...,,послушай я очень устать и мочь сказать что д...
9,18848,18849,2023-12-20 16:47:13+00:00,так а проект тут причем,None,так а проект тут причем,так а проект тут причем,,так а проект тут причем


In [ ]:
comments["lemmas"] = comments["lemmatized_ru_clean"] + comments["lemmatized_en_clean"]
comments.head(10)

,reply_to_msg_id,comment_id,date,text,reactions,lemmatized_ru,lemmatized_en,lemmatized_en_clean,lemmatized_ru_clean,lemmas
0,18859,18861,2023-12-22 06:36:16+00:00,хреновая подборка\njson в csv надёжнее залить ...,None,хреновый подборка \n json в csv надёжнее залит...,хреновая подборка \n json в csv надёжнее залит...,\n json csv csvwriter pandas \n passwo...,хреновый подборка \n в надёжнее залит...,хреновый подборка \n в надёжнее залит...
1,18859,18860,2023-12-22 06:25:35+00:00,скажите ей уже наконец,MessageReactions(results=[ReactionCount(reacti...,сказать ей уже наконец,скажите ей уже наконец,,сказать ей уже наконец,сказать ей уже наконец
2,18809,18851,2023-12-20 19:20:06+00:00,обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,обожать его лекция,обожаю его лекции,,обожать его лекция,обожать его лекция
3,18809,18846,2023-12-20 16:43:20+00:00,объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,объяснять на палец,объясняет на пальцах,,объяснять на палец,объяснять на палец
4,18809,18814,2023-12-20 15:29:01+00:00,это классика,None,это классика,это классика,,это классика,это классика
5,18853,18854,2023-12-21 05:25:45+00:00,не ну такое вполне возможно просто именно для ...,None,не ну такой вполне возможный просто именно для...,не ну такое вполне возможно просто именно для ...,...,не ну такой вполне возможный просто именно для...,не ну такой вполне возможный просто именно для...
6,18852,18853,2023-12-21 05:14:34+00:00,оформление самого гитхаба подписки на проекты ...,None,оформление самого гитхаба подписка на проект и...,оформление самого гитхаба подписки на проекты ...,\n 6 ...,оформление самого гитхаба подписка на проект и...,оформление самого гитхаба подписка на проект и...
7,18825,18852,2023-12-21 04:41:56+00:00,почему за 6 лет работы должно быть чтото на ги...,MessageReactions(results=[ReactionCount(reacti...,почему за 6 год работа должный быть чтото на г...,почему за 6 лет работы должно быть чтото на ги...,6 \n\n selfhoste ...,почему за 6 год работа должный быть чтото на г...,почему за 6 год работа должный быть чтото на г...
8,18795,18850,2023-12-20 16:49:46+00:00,послушай я очень устал и могу сказать что для...,None,послушай я очень устать и мочь сказать что д...,послушай я очень устал и могу сказать что дл...,,послушай я очень устать и мочь сказать что д...,послушай я очень устать и мочь сказать что д...
9,18848,18849,2023-12-20 16:47:13+00:00,так а проект тут причем,None,так а проект тут причем,так а проект тут причем,,так а проект тут причем,так а проект тут причем


In [ ]:
comments['lemmas'] = comments['lemmas'].str.replace(r'\d{1,100}', '', regex=True)

In [ ]:
count_vectorizer = CountVectorizer()
bag_of_words = count_vectorizer.fit_transform(comments['lemmas'])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), columns = count_vectorizer.get_feature_names_out(), index = comments.index)

In [ ]:
a = comments.join(bag_of_words, lsuffix='_bow')

In [ ]:
cols = [
  '👍',
  '⚡',
  '❤',
  '🍌',
  '🎉',
  '🐳',
  '👌',
  '👎',
  '👏',
  '💋',
  '💩',
  '💯',
  '🔥',
  '🗿',
  '🤔',
  '🤡',
  '🤣',
  '🤨',
  '🤬',
  '🤮',
  '🤯',
  '🥰',
  '🥱',
  '🥴',
  '🫡',
  '😁',
  '😐',
  '😱',
  'lemmatized_ru',
  'lemmatized_en',
  'lemmatized_en_clean',
  'lemmatized_ru_clean',
  'lemmas'
]

In [ ]:
a.drop(cols, axis=1, inplace=True, errors='ignore')

In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Columns: 8140 entries, reply_to_msg_id to їх
dtypes: int64(8137), object(3)
memory usage: 220.3+ MB


In [ ]:
df = a.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Columns: 8140 entries, reply_to_msg_id to їх
dtypes: int64(8137), object(3)
memory usage: 220.3+ MB


In [ ]:
# Вычистим маты и не несущие смысла слова (частицы, предлоги и проч.)
url = 'https://github.com/fitwist/telegram-channel-popularity-analysis/blob/main/stopwords.txt'
res = req.get(url)

stopwords = res.content.decode('utf-8')
d = json.loads(stopwords)
words = d['payload']['blob']['rawLines']

df_wout_stopwords = df.drop(words, errors='ignore', axis=1)
df_wout_stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Columns: 7594 entries, reply_to_msg_id to їх
dtypes: int64(7591), object(3)
memory usage: 205.6+ MB


In [ ]:
# Подсчитаем частоты слов
df_wout_stopwords.loc['total'] = df_wout_stopwords.sum(numeric_only=True)

In [ ]:
totals = df_wout_stopwords[df_wout_stopwords['reply_to_msg_id']==57974755]
# df_wout_stopwords.tail()

In [ ]:
# Вычислим самое частое слово (в комментариях)
transposed_totals = totals.transpose()
transposed_totals = transposed_totals.drop(transposed_totals[transposed_totals['total'] == 57974755].index)
transposed_totals = transposed_totals.drop(transposed_totals[transposed_totals['total'] == 58060676].index)
sorted_totals = transposed_totals.sort_values(by=['total'], ascending=False)
sorted_totals.head(20)

,total
код,209.0
питон,167.0
return,158.0
хороший,132.0
def,131.0
язык,124.0
инструмент,123.0
канал,121.0
работать,117.0
написать,114.0


In [ ]:
print(sorted_totals[20:40])

                                total
список                           79.0
пример                           79.0
программирование                 77.0
вопрос                           77.0
работа                           76.0
понять                           74.0
новый                            73.0
время                            71.0
prog_toolshttpstmerkcgoexdumogy  70.0
строка                           69.0
бот                              69.0
пост                             68.0
число                            67.0
сказать                          67.0
огромный                         66.0
понимать                         65.0
задача                           63.0
genius_paulo                     62.0
говорить                         61.0
result                           60.0


# Cамые активные подписчики

In [ ]:
# Подгрузим посты
table = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Ftoe22_tChMfIQHAGFwC1JLFOQjniTKgb6JnN5I9PvI/edit#gid=727621112')
sheet = table.sheet1
comments = pd.DataFrame(sheet.get_all_records())
comments.head()

,reply_to_msg_id,comment_id,date,text,reactions,from_id
0,18863,18864,2023-12-22 18:30:34+00:00,Прошел несколько курсов по Python на Stepik. З...,MessageReactions(results=[ReactionCount(reacti...,412090491
1,18859,18861,2023-12-22 06:36:16+00:00,Хреновая подборка.\nJSON в CSV надёжнее залить...,None,206083161
2,18859,18860,2023-12-22 06:25:35+00:00,"скажите ей уже, наконец",MessageReactions(results=[ReactionCount(reacti...,6269873125
3,18809,18851,2023-12-20 19:20:06+00:00,Обожаю его лекции,MessageReactions(results=[ReactionCount(reacti...,911940866
4,18809,18846,2023-12-20 16:43:20+00:00,Объясняет на пальцах,MessageReactions(results=[ReactionCount(reacti...,617458768


In [ ]:
n = 6
most_active_users = comments['from_id'].value_counts()[:n].index.tolist()
most_active_users

[5508719754, 231727683, 318623145, 67498036, 182806854, 152414494]

user_id, комментариев и реакций, username
*   5508719754 - 126 - @impreza555
*   231727683 - 109 — @Trizalio
*   318623145 - 88 - бот Типичного
*   67498036 - 81 - @StSav012
*   182806854 - 85 - опять @impreza555
*   152414494 - 61 - @D_vVv_N